## Instalaciones

In [2]:
!pip install locationtagger
!pip install geopy
!pip install spacy
!pip install geonamescache

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 41.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 101.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13541 sha256=5951c30fe3fb37329faf7e4

## Importaciones

In [3]:
import pandas as pd
import spacy
import time
from geopy.geocoders import Nominatim
from geonamescache import GeonamesCache
import numpy as np
import math
import locationtagger
import nltk
nltk.downloader.download('maxent_ne_chunker')
nltk.downloader.download('words')
nltk.downloader.download('treebank')
nltk.downloader.download('maxent_treebank_pos_tagger')
nltk.downloader.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/maxent_treebank_pos_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

## Carga de datos

In [4]:
from google.colab import drive
drive.mount('/content/drive')
df_inicial = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Half_Ironman_df6.csv')
df_inicial

Mounted at /content/drive


,Gender,AgeGroup,AgeBand,Country,CountryISO2,EventYear,EventLocation,SwimTime,Transition1Time,BikeTime,Transition2Time,RunTime,FinishTime
0,M,40-44,40,Andorra,AD,2019,IRONMAN 70.3 South American Championship Bueno...,1679,119,9107,95,5515,16514
1,M,45-49,45,Andorra,AD,2019,IRONMAN 70.3 South American Championship Bueno...,2070,177,9160,132,6070,17609
2,M,45-49,45,Andorra,AD,2020,IRONMAN 70.3 Bariloche,1667,161,9891,122,5190,17031
3,M,45-49,45,Andorra,AD,2019,IRONMAN 70.3 World Championship,1750,183,10363,160,5071,17527
4,M,40-44,40,Andorra,AD,2019,IRONMAN 70.3 World Championship,2063,182,10065,142,5556,18008
...,...,...,...,...,...,...,...,...,...,...,...,...,...
840070,M,50-54,50,Zimbabwe,ZW,2015,IRONMAN 70.3 South Africa,2054,261,10527,160,6070,19072
840071,M,40-44,40,Zimbabwe,ZW,2015,IRONMAN 70.3 South Africa,2449,352,11866,265,8461,23393
840072,F,30-34,30,Zimbabwe,ZW,2015,IRONMAN 70.3 Steelhead,2171,357,11433,332,7754,22047
840073,F,35-39,35,Zimbabwe,ZW,2015,IRONMAN 70.3 Budapest,2100,193,10280,233,6148,18954


## Se carga ciudad en nueva columna

In [5]:
# Extracción de los nombres de los eventos
eventos = df_inicial['EventLocation'].unique()
eventos

array(['IRONMAN 70.3 South American Championship Buenos Aires',
       'IRONMAN 70.3 Bariloche', 'IRONMAN 70.3 World Championship',
       'IRONMAN 70.3 Victoria', 'IRONMAN 70.3 Marrakech',
       'IRONMAN 70.3 Santa Cruz', 'IRONMAN 70.3 Dublin',
       'IRONMAN 70.3 Taiwan', 'IRONMAN 70.3 Texas',
       'IRONMAN 70.3 Mallorca', 'IRONMAN 70.3 Barcelona',
       'IRONMAN 70.3 Vichy', 'IRONMAN 70.3 Budapest',
       'IRONMAN 70.3 Punta del Este', 'IRONMAN 70.3 Puerto Rico',
       'IRONMAN 70.3 Liuzhou', 'IRONMAN 70.3 UK',
       "IRONMAN 70.3 Coeur d'Alene", 'IRONMAN 70.3 Wisconsin',
       'IRONMAN 70.3 Cartagena', 'IRONMAN 70.3 Marbella',
       'IRONMAN 70.3 European Championship Elsinore',
       'IRONMAN 70.3 Hawaii', 'IRONMAN 70.3 Boulder',
       'IRONMAN 70.3 Cascais Portugal', 'IRONMAN 70.3 Dubai',
       'IRONMAN 70.3 Nice', 'IRONMAN 70.3 Lanzarote',
       "IRONMAN 70.3 Pays D'Aix", 'IRONMAN 70.3 San Juan',
       'IRONMAN 70.3 Switzerland', 'IRONMAN 70.3 European Championshi

### Se define una función para obtener la ciudad a partir del nombre del evento.

In [6]:
nlp = spacy.load("en_core_web_sm")

gc = GeonamesCache() #Instancia de GeonamesCache para obtener las ciudades a patir textos

ciudades = gc.get_cities()
nombres_ciudad = [city['name'] for city in ciudades.values()]

def obtener_lugar(texto):
  '''Obtiene ciudades a partir de un texto,
     se utilizara para extraer las ciudades
     del nombre de los eventos.'''
  lugar_entidad = locationtagger.find_locations(text = texto)
  ciudades = lugar_entidad.cities
  regiones = lugar_entidad.regions
  paises = lugar_entidad.countries
  if len(ciudades) > 0:
    return ciudades[0]
  if len(regiones) > 0:
    return regiones[0]
  if len(paises) > 0:
    return paises[0]
  doc = nlp(texto)
  for ent in doc.ents:
    if ent.label_ == "GPE":
      ciudad = ent.text
      if ciudad != '':
        return ciudad
  for ciudad in nombres_ciudad:
    if ciudad in texto.split(' '):
      return ciudad

In [7]:
eventos_con_lugar = {}

for evento in eventos:
  eventos_con_lugar[evento] = obtener_lugar(evento)

#### Los diccionarios no tienen método head, se realiza el siguiente paso para visualizar algunos datos.

In [8]:
n_max = 10
primeros_eventos = {k: eventos_con_lugar[k] for i, k in enumerate(eventos_con_lugar) if i < n_max}

for clave, valor in primeros_eventos.items():
    print(clave,"||| Ciudad --->" ,valor)

IRONMAN 70.3 South American Championship Buenos Aires ||| Ciudad ---> Buenos Aires
IRONMAN 70.3 Bariloche ||| Ciudad ---> None
IRONMAN 70.3 World Championship ||| Ciudad ---> None
IRONMAN 70.3 Victoria ||| Ciudad ---> Victoria
IRONMAN 70.3 Marrakech ||| Ciudad ---> None
IRONMAN 70.3 Santa Cruz ||| Ciudad ---> Santa Cruz
IRONMAN 70.3 Dublin ||| Ciudad ---> Dublin
IRONMAN 70.3 Taiwan ||| Ciudad ---> Taiwan
IRONMAN 70.3 Texas ||| Ciudad ---> Texas
IRONMAN 70.3 Mallorca ||| Ciudad ---> None


#### Se observa que quedan algunos eventos sin ciudad.

### Se agrega el Lugar al dataframe con el resto de datos.

In [9]:
df_con_lugar = df_inicial.copy()
df_con_lugar['Lugar'] = df_con_lugar['EventLocation'].map(eventos_con_lugar)

# Eventos con lugar es un diccionario donde la key es EventLocation y el value es la ciudad
# La funcióm map asigna a la columna Lugar, la ciudad asociada al EventLocation que es clave.
# Es equivalente a que eventos_con_lugar sea un dataframe con dos columnas, EventLocation y ciudad
# y hacer un join con df_con_lugar por la clave EventLocation.

### Se agrega manualmente el lugar del evento, para los cuales la función obtener_lugar no devolvio nada.

In [10]:
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Bariloche', 'Lugar'] = 'San Carlos de Bariloche'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Marrakech', 'Lugar'] = 'Marrakech'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Mallorca', 'Lugar'] = 'Mallorca'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Middle East Championship Bahrain', 'Lugar'] = 'Bahrain'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Gulf Coast', 'Lugar'] = 'Gulf Coast'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Eagleman', 'Lugar'] = 'Maryland'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Steelhead', 'Lugar'] = 'Michigan'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 European Championship Elsinore', 'Lugar'] = 'Elsinore'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Lanzarote', 'Lugar'] = 'Lanzarote'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Ruegen', 'Lugar'] = 'Ruegen'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Lanzarote', 'Lugar'] = 'Lanzarote'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Indian Wells La Quinta', 'Lugar'] = 'Indian Wells'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Staffordshire', 'Lugar'] = 'Staffordshire'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Zell am See-Kaprun', 'Lugar'] = 'Kaprun'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Timberman', 'Lugar'] = 'Laconia'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Muskoka', 'Lugar'] = 'Huntsville'
df_con_lugar.loc[df_con_lugar['EventLocation'] == "IRONMAN 70.3 Pays D'Aix", 'Lugar'] = 'Aix-En-Provence'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Sunshine Coast', 'Lugar'] = 'Mooloolaba'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Kronborg', 'Lugar'] = 'Elsinore'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Bintan', 'Lugar'] = 'Bintan'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Silverman', 'Lugar'] = 'Nevada'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Bahrain', 'Lugar'] = 'Bahrain'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Zell am See', 'Lugar'] = 'Kaprun'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Mont-Tremblant', 'Lugar'] = 'Mont-Tremblant'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Cancun', 'Lugar'] = 'Cancun'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN Lake Tahoe', 'Lugar'] = 'Lake Tahoe'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Vineman', 'Lugar'] = 'Guerneville'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Mooseman', 'Lugar'] = 'Newfound Lake'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Superfrog', 'Lugar'] = 'Imperial Beach'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Kraichgau', 'Lugar'] = 'Kraichgau'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Emilia Romagna', 'Lugar'] = 'Cervia'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Bangsaen', 'Lugar'] = 'Bangsaen'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Zell am See Kaprun', 'Lugar'] = 'Kaprun'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Los Cabos', 'Lugar'] = 'Los Cabos'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Alagoas', 'Lugar'] = 'Maceió'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Langkawi', 'Lugar'] = 'Langkawi'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Foz do IguaÃ§u', 'Lugar'] = 'Foz Do Iguaçu'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Dun Laoghaire', 'Lugar'] = 'Dublin'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Florianopolis', 'Lugar'] = 'Florianopolis'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Jonkoping', 'Lugar'] = 'Jönköping'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Maceio', 'Lugar'] = 'Maceió'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Korea', 'Lugar'] = 'Goseong'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Otepaa', 'Lugar'] = 'Otepää'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Pocono Mountains', 'Lugar'] = 'Stroudsburg'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Jeju', 'Lugar'] = 'Jeju-do'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Asia-Pacific', 'Lugar'] = 'Phuket'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Asia-Pacific Championship', 'Lugar'] = 'Cebú'
df_con_lugar.loc[df_con_lugar['EventLocation'] == 'IRONMAN 70.3 Pan American Pro Championship Panama', 'Lugar'] = 'Panamá'

# Campeonatos europeos
df_con_lugar.loc[(df_con_lugar['EventLocation'] == 'IRONMAN 70.3 European Championship') & (df_con_lugar['EventYear'] == 2011), 'Lugar'] = 'Wiesbaden'
df_con_lugar.loc[(df_con_lugar['EventLocation'] == 'IRONMAN 70.3 European Championship') & (df_con_lugar['EventYear'] == 2012), 'Lugar'] = 'Wiesbaden'
df_con_lugar.loc[(df_con_lugar['EventLocation'] == 'IRONMAN 70.3 European Championship') & (df_con_lugar['EventYear'] == 2013), 'Lugar'] = 'Wiesbaden'
df_con_lugar.loc[(df_con_lugar['EventLocation'] == 'IRONMAN 70.3 European Championship') & (df_con_lugar['EventYear'] == 2014), 'Lugar'] = 'Wiesbaden'

# Campeonatos mundiales
df_con_lugar.loc[(df_con_lugar['EventLocation'] == 'IRONMAN 70.3 World Championship') & (df_con_lugar['EventYear'] == 2006), 'Lugar'] = 'Clearwater'
df_con_lugar.loc[(df_con_lugar['EventLocation'] == 'IRONMAN 70.3 World Championship') & (df_con_lugar['EventYear'] == 2009), 'Lugar'] = 'Clearwater'
df_con_lugar.loc[(df_con_lugar['EventLocation'] == 'IRONMAN 70.3 World Championship') & (df_con_lugar['EventYear'] == 2011), 'Lugar'] = 'Las Vegas'
df_con_lugar.loc[(df_con_lugar['EventLocation'] == 'IRONMAN 70.3 World Championship') & (df_con_lugar['EventYear'] == 2012), 'Lugar'] = 'Las Vegas'
df_con_lugar.loc[(df_con_lugar['EventLocation'] == 'IRONMAN 70.3 World Championship') & (df_con_lugar['EventYear'] == 2013), 'Lugar'] = 'Las Vegas'
df_con_lugar.loc[(df_con_lugar['EventLocation'] == 'IRONMAN 70.3 World Championship') & (df_con_lugar['EventYear'] == 2014), 'Lugar'] = 'Mont-Tremblant'
df_con_lugar.loc[(df_con_lugar['EventLocation'] == 'IRONMAN 70.3 World Championship') & (df_con_lugar['EventYear'] == 2015), 'Lugar'] = 'Kaprun'
df_con_lugar.loc[(df_con_lugar['EventLocation'] == 'IRONMAN 70.3 World Championship') & (df_con_lugar['EventYear'] == 2016), 'Lugar'] = 'Mooloolaba'
df_con_lugar.loc[(df_con_lugar['EventLocation'] == 'IRONMAN 70.3 World Championship') & (df_con_lugar['EventYear'] == 2017), 'Lugar'] = 'Chattanooga'
df_con_lugar.loc[(df_con_lugar['EventLocation'] == 'IRONMAN 70.3 World Championship') & (df_con_lugar['EventYear'] == 2018), 'Lugar'] = 'Nelson Mandela Bay'
df_con_lugar.loc[(df_con_lugar['EventLocation'] == 'IRONMAN 70.3 World Championship') & (df_con_lugar['EventYear'] == 2019), 'Lugar'] = 'Nice'

In [11]:
df_con_lugar.head()

,Gender,AgeGroup,AgeBand,Country,CountryISO2,EventYear,EventLocation,SwimTime,Transition1Time,BikeTime,Transition2Time,RunTime,FinishTime,Lugar
0,M,40-44,40,Andorra,AD,2019,IRONMAN 70.3 South American Championship Bueno...,1679,119,9107,95,5515,16514,Buenos Aires
1,M,45-49,45,Andorra,AD,2019,IRONMAN 70.3 South American Championship Bueno...,2070,177,9160,132,6070,17609,Buenos Aires
2,M,45-49,45,Andorra,AD,2020,IRONMAN 70.3 Bariloche,1667,161,9891,122,5190,17031,San Carlos de Bariloche
3,M,45-49,45,Andorra,AD,2019,IRONMAN 70.3 World Championship,1750,183,10363,160,5071,17527,Nice
4,M,40-44,40,Andorra,AD,2019,IRONMAN 70.3 World Championship,2063,182,10065,142,5556,18008,Nice


## Función que obtiene la Longitud y la Latitud del lugar del evento.

In [39]:
# Inicialización del geocoder
geolocator = Nominatim(user_agent="my_geocoder")

def get_latitud_longitud(lugar):
  '''Devuelve latitud y longitud de un lugar'''
  try:
    location = geolocator.geocode(lugar)
    return location.latitude, location.longitude
  except:
  # Si no hay lugar, devuelve nan nan
    return np.nan, np.nan

In [13]:
lugares = set(df_con_lugar['Lugar'])
lugares_con_latitud_longitud = {}
for lugar in lugares:
  lugares_con_latitud_longitud[lugar] = get_latitud_longitud(lugar)

# lugares_con_latitud_longitud es un diccionario, donde la key es el lugar del evento y el value la latidud y longitud

## Se carga latitud y longitud

In [14]:
df_con_lugar['Latitud'] = df_con_lugar['Lugar'].map(lambda lugar: lugares_con_latitud_longitud[lugar][0])
df_con_lugar['Longitud'] = df_con_lugar['Lugar'].map(lambda lugar: lugares_con_latitud_longitud[lugar][1])
df_con_lugar

,Gender,AgeGroup,AgeBand,Country,CountryISO2,EventYear,EventLocation,SwimTime,Transition1Time,BikeTime,Transition2Time,RunTime,FinishTime,Lugar,Latitud,Longitud
0,M,40-44,40,Andorra,AD,2019,IRONMAN 70.3 South American Championship Bueno...,1679,119,9107,95,5515,16514,Buenos Aires,-34.607568,-58.437089
1,M,45-49,45,Andorra,AD,2019,IRONMAN 70.3 South American Championship Bueno...,2070,177,9160,132,6070,17609,Buenos Aires,-34.607568,-58.437089
2,M,45-49,45,Andorra,AD,2020,IRONMAN 70.3 Bariloche,1667,161,9891,122,5190,17031,San Carlos de Bariloche,-41.133442,-71.309843
3,M,45-49,45,Andorra,AD,2019,IRONMAN 70.3 World Championship,1750,183,10363,160,5071,17527,Nice,43.700936,7.268391
4,M,40-44,40,Andorra,AD,2019,IRONMAN 70.3 World Championship,2063,182,10065,142,5556,18008,Nice,43.700936,7.268391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
840070,M,50-54,50,Zimbabwe,ZW,2015,IRONMAN 70.3 South Africa,2054,261,10527,160,6070,19072,South Africa,-28.816624,24.991639
840071,M,40-44,40,Zimbabwe,ZW,2015,IRONMAN 70.3 South Africa,2449,352,11866,265,8461,23393,South Africa,-28.816624,24.991639
840072,F,30-34,30,Zimbabwe,ZW,2015,IRONMAN 70.3 Steelhead,2171,357,11433,332,7754,22047,Michigan,43.621195,-84.682435
840073,F,35-39,35,Zimbabwe,ZW,2015,IRONMAN 70.3 Budapest,2100,193,10280,233,6148,18954,Budapest,47.497892,19.040161


## Se agrega el país de cada evento

In [15]:
def obtener_pais(latitud, longitud):
    geolocator = Nominatim(user_agent='my_app')
    geolocator.headers['Accept-Language'] = 'en'
    ubicacion = geolocator.reverse(f"{latitud}, {longitud}", exactly_one=True)
    if ubicacion:
        direccion = ubicacion.raw['address']
        pais = direccion.get('country')
        return pais
    else:
        return None

#### Se crea un nuevo dataframe que contenga las Latitudes y Longitudes, sin duplicados.

In [16]:
ciudades = df_con_lugar[['Latitud','Longitud']].drop_duplicates()

ciudades.reset_index(inplace= True)

In [17]:
paises = {}

for ciudad in range(len(ciudades)):
  paises[ciudades.loc[ciudad, 'Latitud']] = obtener_pais(ciudades.loc[ciudad, 'Latitud'], ciudades.loc[ciudad, 'Longitud'])

#### Se agrega el país.

In [18]:
df_con_lugar['Pais'] = df_con_lugar['Latitud'].map(lambda pais: paises[pais])

In [19]:
df_con_lugar.to_csv('/content/drive/MyDrive/Colab Notebooks/df_con_cupos.csv', index = False)

### Se crea un dataframe con los datos de los eventos, sin duplicados para poder generar los nodos de la base de datos.

In [20]:
lugares = df_con_lugar[['EventLocation', 'Lugar', 'Latitud', 'Longitud', 'Pais', 'EventYear']].drop_duplicates()

In [21]:
lugares.reset_index(inplace= True)

In [22]:
lugares['EsMundial'] = lugares['EventLocation'] == 'IRONMAN 70.3 World Championship'
lugares.to_csv('/content/drive/MyDrive/Colab Notebooks/ciudades_paises.csv', index = True)

### Se crea un dataframe con los paises, tanto de los eventos como de los participantes

In [23]:
paises_competidores = df_con_lugar['Country'].drop_duplicates().to_frame()
paises_competidores.reset_index(inplace = True)

In [24]:
paises_competidores

,index,Country
0,0,Andorra
1,114,United Arab Emirates
2,559,Afghanistan
3,592,Antigua and Barbuda
4,600,Anguilla
...,...,...
235,821579,Yemen
236,821583,Mayotte
237,821585,South Africa
238,839961,Zambia


In [25]:
paises_competencias = df_con_lugar['Pais'].drop_duplicates().to_frame()
paises_competencias.reset_index(inplace = True)

In [26]:
# Necesario para crear los nodos de paises que no son de eventos.
paises_que_faltan = paises_competidores['Country'][~paises_competidores['Country'].isin(paises_competencias['Pais'])].dropna()

In [27]:
paises_que_faltan

0                  Andorra
2              Afghanistan
3      Antigua and Barbuda
4                 Anguilla
5                  Albania
              ...         
234                  Samoa
235                  Yemen
236                Mayotte
238                 Zambia
239               Zimbabwe
Name: Country, Length: 189, dtype: object

In [28]:
paises_que_faltan.to_csv('/content/drive/MyDrive/Colab Notebooks/paises_faltantes.csv', index = False)

In [29]:
paises_competidores

,index,Country
0,0,Andorra
1,114,United Arab Emirates
2,559,Afghanistan
3,592,Antigua and Barbuda
4,600,Anguilla
...,...,...
235,821579,Yemen
236,821583,Mayotte
237,821585,South Africa
238,839961,Zambia


In [30]:
paises_totales = set(pd.concat([paises_competencias, pd.DataFrame({'Pais': paises_que_faltan})])['Pais'])
paises_totales

{'Afghanistan',
 'Aland Islands',
 'Albania',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Angola',
 'Anguilla',
 'Antarctica',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bonaire, Saint Eustatius and Saba',
 'Bosnia and Herzegovina',
 'Botswana',
 'Bouvet Island',
 'Brazil',
 'British Indian Ocean Territory',
 'British Virgin Islands',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Christmas Island',
 'Cocos (Keeling) Islands',
 'Colombia',
 'Comoros',
 'Congo Republic',
 'Cook Islands',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cuba',
 'Curacao',
 'Cyprus',
 'Czech Republic',
 'DR Congo',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic

In [40]:
paises_con_latitud_longitud = {}
for pais in paises_totales:
  paises_con_latitud_longitud[pais] = get_latitud_longitud(pais)
paises_con_latitud_longitud

Norfolk Island
Philippines
San Marino
Costa Rica
Indonesia
Latvia
New Caledonia
Türkiye
Mayotte
Lebanon
Greenland
Mexico
Malta
Bhutan
South Africa
Central African Republic
Peru
Falkland Islands
China
United Arab Emirates
Cyprus
Serbia
South Korea
Norway
French Southern Territories
Venezuela
British Virgin Islands
Sierra Leone
Panama
Syria
Chad
Fiji
United States Virgin Islands
Algeria
Somalia
Turkmenistan
Malaysia
Azerbaijan
Finland
Botswana
Brazil
French Guiana
France
Israel
Laos
Italy
Sweden
Jordan
Tunisia
Macau
Lithuania
American Samoa
Romania
Suriname
Greece
Spain
Nicaragua
Benin
Monaco
Northern Mariana Islands
Canada
United States Minor Outlying Islands
Dominica
Bahrain
Equatorial Guinea
Antarctica
Austria
Egypt
Yemen
Saudi Arabia
Turks and Caicos Islands
Anguilla
Belgium
Western Sahara
Poland
Vietnam
Guatemala
Croatia
Bulgaria
Colombia
Seychelles
Congo Republic
Turkey
North Macedonia
Martinique
Antigua and Barbuda
Bouvet Island
Belize
Andorra
Liberia
Haiti
Afghanistan
Singapore
S

{'Norfolk Island': (-29.0328038, 167.9483137),
 'Philippines': (12.7503486, 122.7312101),
 'San Marino': (43.9458623, 12.458306),
 'Costa Rica': (10.2735633, -84.0739102),
 'Indonesia': (-2.4833826, 117.8902853),
 'Latvia': (56.8406494, 24.7537645),
 'New Caledonia': (-21.3019905, 165.4880773),
 'Türkiye': (38.9597594, 34.9249653),
 'Mayotte': (-12.825543, 45.148490830203045),
 'Lebanon': (33.8750629, 35.843409),
 'Greenland': (77.6192349, -42.8125967),
 'Mexico': (23.6585116, -102.0077097),
 'Malta': (35.8885993, 14.4476911),
 'Bhutan': (27.549511, 90.5119273),
 'South Africa': (-28.8166236, 24.991639),
 'Central African Republic': (7.0323598, 19.9981227),
 'Peru': (-6.8699697, -75.0458515),
 'Falkland Islands': (-51.9492937, -59.5383657),
 'China': (35.000074, 104.999927),
 'United Arab Emirates': (24.0002488, 53.9994829),
 'Cyprus': (34.9823018, 33.1451285),
 'Serbia': (44.1534121, 20.55144),
 'South Korea': (36.638392, 127.6961188),
 'Norway': (61.1529386, 8.7876653),
 'French Sout

In [45]:
df_paises = pd.DataFrame.from_dict(paises_con_latitud_longitud, orient='index', columns=['Latitud', 'Longitud']).reset_index()
df_paises.columns = ['Nombre_Pais', 'Latitud', 'Longitud']

df_paises.to_csv('/content/drive/MyDrive/Colab Notebooks/paises_lat_long.csv', index=False)
print(df_paises)

             Nombre_Pais    Latitud    Longitud
0         Norfolk Island -29.032804  167.948314
1            Philippines  12.750349  122.731210
2             San Marino  43.945862   12.458306
3             Costa Rica  10.273563  -84.073910
4              Indonesia  -2.483383  117.890285
..                   ...        ...         ...
236           Mozambique -19.302233   34.914498
237                Niger  17.735621    9.323843
238              Vanuatu -16.525507  168.106915
239              Comoros -12.204518   44.283296
240  Trinidad and Tobago  10.746690  -61.084007

[241 rows x 3 columns]
